### Construct numpy array for whole training dataset using the uniformizing techniques

* SSS
* ESS
* SIZ

In [ ]:
!pip install SimpleITK # for NIFTI file conversion to numpy array
!pip install tqdm # for seeing progress

#### ATTENTION

Put all NIFTI files from TrainingSet_2_of_2 to TrainingSet_1_of_2, you should get 218 NIFTI files

In [ ]:
import SimpleITK as sitk
import os
from tqdm import tqdm
import numpy as np 
import pandas as pd
import os
import scipy.ndimage
import matplotlib.pyplot as plt
from skimage import measure, morphology
import cv2
from scipy.ndimage import zoom
%matplotlib inline


np.random.seed(42)

# Helpers functions

def create_directory(directory):
    '''
    Creates a new folder in the specified directory if the folder doesn't exist.
    INPUT
        directory: Folder to be created, called as "folder/".
    OUTPUT
        New folder in the current directory.
    '''
    if not os.path.exists(directory):
        os.makedirs(directory)


# Define paths
base_path = os.path.abspath("./") # Your root directory
dataset_path = os.path.join(base_path, "dataset") # Your dataset folder
model_path = os.path.join(base_path, "models")
log_path = os.path.join(base_path, "logs")

# Raw data path
INPUT_FOLDER = 'TrainingSet_1_of_2/'  
training_data_raw = os.path.join(dataset_path, INPUT_FOLDER)

# Check inside target folder
print(os.listdir(dataset_path))
print(training_data_raw)

patients = sorted(os.listdir(training_data_raw))
print(patients[:10])

In [ ]:
# Resize 2D slices
w, h = 128, 128
def rs_img(img):
    '''W and H is 128 now
    '''
    img = np.transpose(img)
    flatten = [cv2.resize(img[:,:,i], (w, h), interpolation=cv2.INTER_CUBIC) for i in range(img.shape[-1])]
    img = np.array(np.dstack(flatten)) 
    return img


# Subset slice selection (SSS)
def change_depth_sss(img):
    
    factor = 16
    img_start = img[:,:,:factor]
    
    mid = int(img.shape[-1]/2)
    img_middle = img[:,:,mid-factor:mid+factor]
    
    img_end = img[:,:,-factor:]
    img = np.concatenate((img_start, img_middle, img_end), axis=2)
    return img


# Even slice selection (ESS)
def change_depth_ess(img):
    
    '''ESS depth size is 32 as min depth size is 47, so we cannot evenly select 64 slices!'''

    target_depth = 32

    #print(img.shape)

    scan_depth = int(img.shape[-1])
    factor = int(np.floor(scan_depth / target_depth)) # min depth is 32

    # floor and ceil result in different values
    
    #print(scan_depth, factor)
    
    flatten = []
    idx = 0

    for i in range(0, scan_depth, factor):
      #print(idx)
    
        if idx>=scan_depth:
            break
        else:
            flatten.append(img[:,:,i])

    img = np.array(np.dstack(flatten))

    # hardcode from bottom, if not 32 slices
    img = img[:,:,:target_depth]

    #print(img.shape)

    assert img.shape[-1] == target_depth , "Error"
    return img


# Spline interpolated zoom (SIZ)
def change_depth_siz(img):
    desired_depth = 64
    current_depth = img.shape[-1]
    depth = current_depth / desired_depth
    depth_factor = 1 / depth
    img_new = zoom(img, (1, 1, depth_factor), mode='nearest')
    return img_new


def normalize(image):
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    return image
    

def zero_center(image):
    image = image - PIXEL_MEAN
    return image


def plot_seq(data, name):

  from numpy.random import rand
  import matplotlib.pyplot as plt
  
  a, b = 3, 10
  data = np.reshape(data, (a, b, 128, 128))
  test_data = data
  r, c = test_data.shape[0], test_data.shape[1]

  cmaps = [['viridis', 'binary'], ['plasma', 'coolwarm'], ['Greens', 'copper']]

  heights = [a[0].shape[0] for a in test_data]
  widths = [a.shape[1] for a in test_data[0]]

  fig_width = 10.  # inches
  fig_height = fig_width * sum(heights) / sum(widths)

  f, axarr = plt.subplots(r,c, figsize=(fig_width, fig_height),
          gridspec_kw={'height_ratios':heights})

  for i in range(r):
      for j in range(c):
          axarr[i, j].imshow(test_data[i][j], cmap='gray')
          axarr[i, j].axis('off')

  plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
  #plt.savefig('{}/{}.png'.format(log_path, name), dpi=1000)
  plt.show()


# NOT USING THIS NOW
def resize_depth_wise(img3d):
    '''
      Inputs a 3d tensor with uneven depth
      Outputs a 3d tensor with even depth, in this case depth=64
    '''
  
  # patient image 3D
    p = img3d
  # list of 3D slices of p
    p_2d = []

    depth = 2

    n = 0
    c = 0

    for c in range(70):
        img = img3d[:,:,n+depth]
        p_2d.append(img)
        n = n+depth
        c = c+1
        
    p_3d_d64 = np.array(np.dstack(p_2d))
    return p_3d_d64


In [ ]:
img = sitk.ReadImage(training_data_raw + patients[5])
img = sitk.GetArrayFromImage(img)
img = rs_img(img)
print("Raw volume size: ", img.shape)

img_ess = change_depth_ess(img)
img_sss = change_depth_sss(img)
img_siz = change_depth_siz(img)

img_sss = np.transpose(img_sss)
img_ess = np.transpose(img_ess)
img_siz = np.transpose(img_siz)

print(img_sss.shape, img_ess.shape, img_siz.shape)


vols = [img_sss[:30], img_ess[:30], img_siz[:30]]
tech = ['sss', 'ess', 'siz']

for v, t in zip(vols, tech):
    plot_seq(v, t)

In [ ]:
# Load training data

depths = []

pts = []

for ps in tqdm(patients[:]):

    #print("Image Name", ps)
    img = sitk.ReadImage(training_data_raw + ps)
    img = sitk.GetArrayFromImage(img)
    
    # Resize x,y,z
    img = rs_img(img)
    
    depths.append(img.shape[-1])

    # What downsampling technique?
    img = change_depth_sss(img)
    
    pts.append(img)
    img=None

# Convert to numpy array
train_data = np.array(pts)
train_data.shape
pts = None
print("Min depth:", min(depths))

In [ ]:
######################################################################
# Test data path
INPUT_FOLDER = 'TestSet/'  
test_data_raw = os.path.join(dataset_path, INPUT_FOLDER)

patients_test = sorted(os.listdir(test_data_raw))
print(len(patients_test))
print(patients_test[:2])

# Load test data
pts_test = []

for ps in tqdm(patients_test[:]):

    #print("Image Name", ps)
    img = sitk.ReadImage(test_data_raw + ps)
    img = sitk.GetArrayFromImage(img)
    
    # Resize x,y,z
    img = rs_img(img)

    # What downsampling technique?
    img = change_depth_sss(img)

    pts_test.append(img)
    img=None

# Convert to numpy array
test_data = np.array(pts_test)
test_data.shape
######################################################################

In [ ]:
# Get these values for preprocessing

MIN_BOUND = np.min(train_data)
MAX_BOUND = np.max(train_data)
PIXEL_MEAN = np.mean(train_data)

print("Min", MIN_BOUND)
print("Min", MAX_BOUND)
print("Mean", PIXEL_MEAN)

In [ ]:
# Normalize
x_train = normalize(train_data)
#x_test = normalize(test_data)

In [ ]:
np.max(x_train), np.min(x_train)

In [ ]:
# Get pixel mean from normalized data

PIXEL_MEAN = np.mean(x_train)
PIXEL_MEAN

# MEAN SIZ = 0.4106
#
#

In [ ]:
# Zero center data
x_train_zero = zero_center(x_train)
#x_test_zero = zero_center(x_test)

In [ ]:
# After processing

MIN_BOUND = np.min(x_train_zero)
MAX_BOUND = np.max(x_train_zero)
PIXEL_MEAN = np.mean(x_train_zero)

print("Min", MIN_BOUND)
print("Max", MAX_BOUND)
print("Mean", PIXEL_MEAN)

In [ ]:
print(x_train_zero.shape) #x_test_zero.shape)

In [ ]:
#plt.imshow(x_test[0][:,:,10])

In [ ]:
#print(x_train_zero.shape, x_test_zero.shape)

In [ ]:
# Save training and test data
#np.save('{}/x_test.npy'.format(dataset_path), x_test)
np.save('{}/x_train_sss.npy'.format(dataset_path), x_train_zero)